# Universality of Single Qubit Gates

## Challenge Statement

In quantum computing, we have two fundamental structures: states and operators, which we can represent with vectors and matrices respectively. <br>

Vectors must have norm 1 (since they determine a sum of probabilities) and matrices must be unitary. <br>
  
When building a quantum computer, we would like it to be able to generate any  
U operator we need. However, we cannot physically implement every possible operator, so the goal is to create subsets of gates or operators that are able to generate all the others. <br>

To do so, you will use a variational method to find these parameters. Additionally, you must build an error function that tells us how close your matrix is to the input unitary U.

In [1]:
import json
import pennylane as qml
import pennylane.numpy as np

In [23]:
def get_matrix_2(params):
    alpha, beta, gamma, phi = params
#    sigma_x = np.array([[0, 1], [1, 0]])
#    sigma_y = np.array([[0, -1j], [1j, 0]])
#    sigma_z = np.array([[1, 0], [0, -1]])

    # Calculate the individual rotation matrices
    rz_gamma = np.array([[np.exp(-1j * gamma / 2), 0], [0, np.exp(1j * gamma / 2)]])
    rx_beta = np.array([[np.cos(beta / 2), -1j * np.sin(beta / 2)],
                        [-1j * np.sin(beta / 2), np.cos(beta / 2)]])
    rz_alpha = np.array([[np.exp(-1j * alpha / 2), 0], [0, np.exp(1j * alpha / 2)]])

    # Combine the rotation matrices in the specified order
    unitary_matrix = np.exp(1j * phi) * rz_gamma @ rx_beta @ rz_alpha

    return unitary_matrix

In [4]:
np.random.seed(1967)

def get_matrix(params):
    """
    Args:
        - params (array): The four parameters of the model.
        
    Returns:
        - (matrix): The associated matrix to these parameters.
    """
    
    alpha, beta, gamma, phi = params

    # Put your code here #

    RZ_gamma = np.array([[np.exp(complex(0,-gamma/2)), 0], 
                         [0, np.exp(complex(0, gamma/2))]])
    
    RX_beta = np.array([[np.cos(beta/2), complex(0,-np.sin(beta/2))], 
                        [complex(0,-np.sin(beta/2)), np.cos(beta/2)]])
    
    RZ_alpha = np.array([[np.exp(complex(0,-alpha/2)), 0], 
                         [0, np.exp(complex(0, alpha/2))]])
    
    
    the_matrix = np.multiply(np.exp(complex(0, phi)), np.matmul(RZ_gamma, np.matmul(RX_beta, RZ_alpha)))

    
     # Calculate the individual rotation matrices
    rz_gamma = np.array([[np.exp(-1j * gamma / 2), 0], [0, np.exp(1j * gamma / 2)]])

    rx_beta = np.array([[np.cos(beta / 2), -1j * np.sin(beta / 2)],
                        [-1j * np.sin(beta / 2), np.cos(beta / 2)]])
    
    rz_alpha = np.array([[np.exp(-1j * alpha / 2), 0], [0, np.exp(1j * alpha / 2)]])

    # Combine the rotation matrices in the specified order
    unitary_matrix = np.exp(1j * phi) * rz_gamma @ rx_beta @ rz_alpha

    
    # Return the matrix
    #return the_matrix
    return unitary_matrix

def error(U, params):
    """
    This function determines the similarity between your generated matrix and
    the target unitary.

    Args:
        - U (np.array): Goal matrix that we want to approach.
        - params (array): The four parameters of the model.

    Returns:
        - (float): Error associated with the quality of the solution.
    """

    matrix = get_matrix(params)

    # Put your code here #

    error_value = np.linalg.norm(U - matrix)
    
    #error = np.sqrt(sum([x**2 for x in (matrix - U).flatten()]))
    
    # Return the error
    return error_value


def train_parameters(U):
    # epochs = 1000
    epochs = 10
    lr = 0.01

    grad = qml.grad(error, argnum=1)
    params = np.random.rand(4) * np.pi

    for epoch in range(epochs):
        print("epoch:", epoch, type(params))
        params -= lr * grad(U, params)

    return params


In [26]:
params = np.random.rand(4) * np.pi
print("monit:", get_matrix_2(params))
print()
print(get_matrix(params))

monit: [[ 0.08516316-0.04481695j  0.61131839+0.78551162j]
 [ 0.91838334-0.38381079j -0.05215583-0.08087702j]]

[[ 0.08516316-0.04481695j  0.61131839+0.78551162j]
 [ 0.91838334-0.38381079j -0.05215583-0.08087702j]]


In [21]:
params = np.random.rand(4) * np.pi
alpha, beta, gamma, phi = params
print("params:", params)
print()
result = get_matrix(params)
print()
x = np.exp(complex(0, phi))

rz_gamma =  [[np.exp(complex(0, -gamma/2)), 0],
            [0, np.exp(complex(0, gamma/2))]] 

rx_beta = [[np.cos(beta/2), complex(0,-np.sin(beta/2))],
           [complex(0,-np.sin(beta/2)), np.cos(beta/2)]]

rz_alpha = [[np.exp(complex(0, -alpha/2)), 0],
            [0, np.exp(complex(0, alpha/2))]]

print("result:", result.shape, result)
print()

rz_rx_rz = np.matmul(rz_gamma, np.matmul(rx_beta, rz_alpha))

print("check:", np.multiply(x, rz_rx_rz))

params: [2.27803033 1.80051172 2.0258912  1.75304041]


result: (2, 2) [[ 0.572617  -0.24137019j  0.74654194+0.23775056j]
 [ 0.78225008+0.0439888j  -0.44895819-0.42963513j]]

check: [[ 0.572617  -0.24137019j  0.74654194+0.23775056j]
 [ 0.78225008+0.0439888j  -0.44895819-0.42963513j]]


In [6]:
get_matrix([.2,.3,.4,.5])

tensor([[0.96906149+0.19643849j, 0.05819395-0.13764163j],
        [0.08437912-0.12333661j, 0.68888344+0.70930096j]], requires_grad=True)

In [7]:
test_input = [[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]
expected_output = [[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]
params = [.2,.3,.4,.5]
error(test_input, params)

<function __main__.error(U, params)>

In [8]:
test_input = [[ 1,  0], [ 0, -1]]
expected_output = [[ 1,  0], [ 0, -1]]
params = [.2,.3,.4,.5]
error(test_input, params)

<function __main__.error(U, params)>

In [ ]:
# train_parameters(test_input)

In [5]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    matrix = json.loads(test_case_input)
    params = [float(p) for p in train_parameters(matrix)]
    return json.dumps(params)


def check(solution_output: str, expected_output: str) -> None:
    matrix1 = get_matrix(json.loads(solution_output))
    matrix2 = json.loads(expected_output)
    print(matrix1)
    assert not np.allclose(get_matrix(np.random.rand(4)), get_matrix(np.random.rand(4)))
    assert np.allclose(matrix1, matrix2, atol=0.2)


# These are the public test cases
test_cases = [
    ('[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]', '[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]'),
    ('[[ 1,  0], [ 0, -1]]', '[[ 1,  0], [ 0, -1]]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]'...
epoch: 0 <class 'pennylane.numpy.tensor.tensor'>
Runtime Error. complex() second argument must be a number, not 'ArrayBox'
Running test case 1 with input '[[ 1,  0], [ 0, -1]]'...
epoch: 0 <class 'pennylane.numpy.tensor.tensor'>
Runtime Error. complex() second argument must be a number, not 'ArrayBox'
